<a href="https://colab.research.google.com/github/steelannelida/nanoGPT/blob/master/nanogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/steelannelida/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 682, done.
remote: Total 682 (delta 0), reused 0 (delta 0), pack-reused 682 (from 1)
Receiving objects: 100% (682/682), 952.47 KiB | 17.97 MiB/s, done.
Resolving deltas: 100% (385/385), done.


In [4]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00


In [9]:
!yes  | python nanoGPT/data/shakespeare/prepare.py

train has 301,966 tokens
val has 36,059 tokens


In [16]:
with open('nanoGPT/data/shakespeare/input.txt') as f:
  text = f.read()

chars = sorted(set(text))
vocab_size=len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}

def encode(t):
  return [stoi[c] for c in t]

def decode(seq):
  return ''.join([chars[i] for i in seq])

decode(encode("sandwitch"))

'sandwitch'

In [20]:
import torch

data = torch.tensor(encode(text))
data.shape, data.dtype

n = int(data.shape[0] * 0.9)
train_data = data[:n]
valid_data = data[n:]

In [32]:
#torch.manual_seed(1337)


def get_batch(data_set=train_data, batch_size=32, seq_length=128):
    x = torch.zeros([batch_size, seq_length], dtype=torch.int)
    y = torch.zeros([batch_size, seq_length], dtype=torch.int)
    for b in range(batch_size):
        t = torch.randint(0, data_set.shape[0] - seq_length - 1, [1])
        x[b] = data_set[t:t+seq_length]
        y[b] = data_set[t+1:t+seq_length+1]
    return x, y

x, y = get_batch()
print(decode(x[13]))
print(decode(y[13]))

nour would become such a person. that it was
no better than picture-like to hang by the wall, if
renown made it not stir, was pl
our would become such a person. that it was
no better than picture-like to hang by the wall, if
renown made it not stir, was ple


In [55]:
import torch.nn as nn

class BGM(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx):
    return self.embeddings(idx)

  def generate(self, idx0, l):
    result = torch.zeros([l+1], dtype=torch.int)
    for i in range(l):
      logits = self.forward(result[i])
      sm = logits.flatten().softmax(0)
      next_idx = torch.multinomial(sm, 1)
      result[i+1] = next_idx
    return result

model = BGM(vocab_size)
decode(model.generate(9, 10))
#model(x).shape

'\nIoWHC3F?Bo'

In [49]:
x, y = get_batch()
logits = model(x)
loss_fun = nn.CrossEntropyLoss()
loss = loss_fun(logits.permute(0, 2, 1), y.long())
loss

tensor(4.6245, device='cuda:0', grad_fn=<NllLoss2DBackward0>)

In [106]:
torch.manual_seed(1337)

model = BGM(vocab_size)
model.cuda()
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
xv, yv = get_batch(valid_data, batch_size=128)
xv, yv = xv.cuda(), yv.cuda()

In [107]:



for step in range(11000):
  x, y = get_batch()
  x, y = x.cuda(), y.cuda()

  logits = model.forward(x)
  loss = loss_fun(logits.permute(0, 2, 1), y.long())
  opt.zero_grad()
  loss.backward()
  opt.step()

  if step % 100 == 0:
    vlogits = model.forward(xv)
    vloss = loss_fun(vlogits.permute(0,2,1), yv.long())
    print('%f\t%f'%(loss, vloss))


4.742974	4.734350
4.593366	4.589903
4.415133	4.451496
4.308665	4.319388
4.183056	4.193480
4.042974	4.073759
3.963645	3.960375
3.836848	3.853101
3.768152	3.751799
3.651063	3.656808
3.543638	3.567621
3.439544	3.483817
3.402012	3.405536
3.292839	3.332559
3.264008	3.264529
3.164066	3.201076
3.115824	3.142321
3.098186	3.088265
3.023422	3.037897
2.980609	2.991637
2.926366	2.948941
2.867295	2.909889
2.841636	2.874276
2.837764	2.841379
2.793632	2.810919
2.785696	2.783268
2.739919	2.757930
2.683762	2.735145
2.668131	2.714252
2.710148	2.695304
2.643085	2.678021
2.644850	2.662347
2.596774	2.647717
2.598882	2.634745
2.609964	2.622900
2.599167	2.612088
2.578844	2.602254
2.559395	2.593364
2.567649	2.585398
2.532679	2.578202
2.557700	2.571599
2.531308	2.565480
2.536976	2.560132
2.517168	2.555228
2.521195	2.550843
2.493500	2.546568
2.504177	2.542770
2.500721	2.539268
2.483087	2.535913
2.508026	2.533042
2.520510	2.530323
2.483397	2.528173
2.484283	2.525993
2.508175	2.523500
2.490886	2.521852
2.446994	2

In [111]:
print(decode(model.cpu().generate(encode('q'), 120)))


Se
Fis d,
TI n:

OME sheve ind f bevemanes kimad'T:
QUCEShaloro llomewileat t.

t lwntelllllen s! m harilet? l y. w tt t
